## Se tiene información estadística de la temporada regular de todos los jugadores de la NBA en un RDD de tuplas con el siguiente formato: (id_jugador, nombre, promedio_puntos, promedio_asistencias, promedio_robos, promedio_bloqueos, promedio_rebotes, promedio_faltas). Un analista de la cadena ESPN está trabajando con un RDD que corresponde a la primera ronda de playoffs y que tiene el siguiente formato: (id_jugador, id_partido, timestamp, cantidad_puntos, cantidad_rebotes, cantidad_bloqueos, cantidad_robos, cantidad_asistencias, cantidad_faltas). En base a estos RDDs se quiere programar en PySpark un programa que genere un RDD con los nombres (sin duplicados) de los jugadores que lograron en algún partido de playoffs una cantidad de asistencias mayor a su promedio histórico.


In [ ]:
#Creamos la Spark Session
spark = SparkSession.builder.getOrCreate()

#Creamos el Spark Context
sc = spark.sparkContext

#Genero los RDD dada una lista
RDDEstadisticas = sc.parallelize(estadisticas, 8)
RDDAnalisis = sc.parallelize(analisis, 8)

In [ ]:
#Me quedo con los campos de los RDD que me importan para el ejercicio.
#(id_jugador, nombre, promedio_asistencias).
EstadisticasTemporada = RDDEstadisticas.map(lambda x: (x[0], x[1], x[3]) )

In [ ]:
#Me quedo con los campos de los RDD que me importan para el ejercicio.
#( (id_jugador, id_partido) , promedio_asistencias).
AnalisisPlayOff = RDDAnalisis.map( lambda x: ((x[0]x[1]),x[7]) )

#Sumo la cantidad de asistencias para cada jugador en un mismo partido.
AnalisisPlayOff = AnalisisPlayOff.reduceByKey(lambda a,b: a+b)

#Arreglo el RDD (id_jugador, asistencias_totales_por_partido).
AnalisisPlayOff = AnalisisPlayOff.map(lambda x: (x[0][0],x[1]))

#El mismo jugador en distintos partidos pudo hacer distintas asistencias me quedo con la mas alta.
#(id_jugador, asistencia_mas_alta_playoff).
AnalisisPlayOff = AnalisisPlayOff.reduceByKey(lambda a,b : a if a > b else b)

In [ ]:
#Junto ambos RDD y obtengo un RDD (id_jugador,(asistencia_mas_alta_playoff, nombre, prom_asistencias)).
BusquedaDeJugador = AnalisisPlayOff.leftOuterJoin(EstadisticasTemporada)

In [ ]:
#Filtro los que cumplen que la asistencia en PlayOff es mayor al promedio historico.
MayorQuePromedio = BusquedaDeJugador.filter(lambda x : x[1][0] > x[1][2])

In [ ]:
#Me quedo con los nombres de los superados.
JugadoresConMayorQuePromedio = MayorQuePromedio.map(lambda x: x[1][1])